In [1]:
import torch
import torch.nn as nn
import os
import sys
from pathlib import Path

# addpath('../')
sys.path.append(os.path.join(Path().resolve(), '../')) # add parent directory to path
import ptlk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
print(torch.cuda.is_available())

True


In [8]:
ptnet = ptlk.pointnet.PointNet_features()
base_model = ptlk.pointlk.PointLK(ptnet=ptnet)

In [9]:
# 事前学習済みの重みを読み込む
load_path = './results/ex1/ex1_pointlk_1122_model_best.pth'
load_weights = torch.load(load_path)
base_model.load_state_dict(load_weights)

<All keys matched successfully>

In [11]:
base_model

PointLK(
  (ptnet): PointNet_features(
    (h1): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (h2): Sequential(
      (0): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
  )
)

In [12]:
import open3d as o3d

point_cloud = o3d.io.read_point_cloud('./dataset/PointCloudSet/chair/test/chair1593.ply')

In [14]:
base_model.eval()
with torch.no_grad():
    pred = base_model(point_cloud)

TypeError: forward() missing 1 required positional argument: 'p1'

In [8]:
# dataloader
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1, shuffle=False)

# カスタムヘッドの追加
custom_head = nn.Sequential(
    nn.Linear(in_features, out_features),  # カスタムヘッドのアーキテクチャを設計
    nn.ReLU(),
    nn.Linear(out_features, num_classes)  # 分類の数に合わせた出力層
)

# 新しいモデルを構築
model = nn.Sequential(
    base_model,
    custom_head
)

learning_rate = 1e-3

# カスタムヘッドのパラメータのみをトレーニング
optimizer = torch.optim.SGD(custom_head.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

# データセットを使用してモデルをトレーニング
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # データをモデルに渡す
        scores = model(data)
        loss = criterion(scores, targets)

        # バックプロパゲーションとパラメータの更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# モデルを保存
torch.save(model.state_dict(), 'my_model.pth')

NameError: name 'testset' is not defined